# Declaration

In [1]:
import keras
import re
import nltk
import numpy as np
from keras import preprocessing
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Model
from keras.layers import SeparableConv1D, MaxPooling1D
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint



keras.__version__

Using TensorFlow backend.


'2.2.4'

# Data Loading

In [2]:
trainFromTextFile = "train.FROM"
trainToTextFile   = "train.TO"
trainFromText     = open(trainFromTextFile, 'r', encoding='utf-8').read().lower()
trainToText       = open(trainToTextFile, 'r', encoding='utf-8').read().lower()
trainFromSentenceTokens = re.split('\n', trainFromText)
trainToSentenceTokens   = re.split('\n', trainToText)
trainFromWordTokens = re.split(' |\n', trainFromText)
trainToWordTokens   = re.split(' |\n', trainToText)
print(trainFromSentenceTokens[1])
print(trainToSentenceTokens[1])
print('Found %s sentences from TrainFrom Text' %len(trainFromSentenceTokens))
print('Found %s sentences from TrainTo Text' %len(trainToSentenceTokens))
print('Found %s words from TrainFrom Text' %len(trainFromWordTokens))
print('Found %s words from TrainTo Text' %len(trainToWordTokens))

is it me, or does 'frank moss' look like a child molester?
to me, it looks more like he 'takes a wide stance' on public restroom sex.
Found 29620 sentences from TrainFrom Text
Found 29620 sentences from TrainTo Text
Found 521666 words from TrainFrom Text
Found 479824 words from TrainTo Text


# Tokenization

In [3]:
max_len = 100    # We will cut reviews after 100 words
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizerData = Tokenizer(num_words=max_words)
tokenizerData.fit_on_texts(trainFromSentenceTokens)
sequences = tokenizerData.texts_to_sequences(trainFromSentenceTokens)

sequences = pad_sequences(sequences, maxlen=max_len)  #Pad so all the arrays are the same size

Dataword_index = tokenizerData.word_index
Dataword_count = tokenizerData.word_counts
nWordsData     = len(tokenizerData.word_counts) + 1


print('Found %s words.' %len(trainFromWordTokens))
print('Found %s sentences.' %len(trainFromSentenceTokens))
print('Found %s.' %len(sequences))
print('Found %s unique tokens.' % len(Dataword_index))
print('Found %s unique words.' % len(Dataword_count))

Found 521666 words.
Found 29620 sentences.
Found 29620.
Found 34823 unique tokens.
Found 34823 unique words.


In [4]:
tokenizerTarget = Tokenizer(num_words=max_words)
tokenizerTarget.fit_on_texts(trainToSentenceTokens)
sequencesTarget = tokenizerTarget.texts_to_sequences(trainToSentenceTokens)

sequencesTarget = pad_sequences(sequencesTarget, maxlen=max_len)  #Pad so all the arrays are the same size

Targetword_index = tokenizerTarget.word_index
Targetword_count = tokenizerTarget.word_counts
nWordsTarget     = len(tokenizerTarget.word_counts) + 1


print('Found %s words.' %len(trainToWordTokens))
print('Found %s sentences.' %len(trainToSentenceTokens))
print('Found %s.' %len(sequencesTarget))
print('Found %s unique tokens.' % len(Targetword_index))
print('Found %s unique words.' % len(Targetword_count))

Found 479824 words.
Found 29620 sentences.
Found 29620.
Found 34369 unique tokens.
Found 34369 unique words.


In [105]:
train_data = sequences
train_target = sequencesTarget
train_target_data = sequencesTarget[:+1]

print(train_data.shape)
print(train_target.shape)

(29620, 100)
(29620, 100)


In [107]:
train_target[10]
train_target_data[9]

IndexError: index 9 is out of bounds for axis 0 with size 0

# Embedding Layer

In [7]:
glove_dir = r'C:\Users\mosta\Desktop\Deep Learning Projects\Projects\Lab Exercises\Machine Learning Projects\glove.6B\glove.6B.300d.txt'

embeddings_index = {}
f = open(glove_dir, encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
# The Embedding layer takes at least two arguments:
# the number of possible tokens, the maximum word index,
# and the dimensionality of the embeddings, here 300.
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in Dataword_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [46]:
embedding_matrix.shape

(10000, 300)

# Model Creation

Encoder / Decoder

In [96]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, len(trainFromSentenceTokens)))
encoder = LSTM(embedding_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, len(trainToSentenceTokens)))
# We set up our decoder to return full output sequences, and to return internal states as well. 
# We don't use the return states in the training model, but we will use them in inference.
decoder = LSTM(embedding_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(train_target), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
grandmodel = Sequential()
grandmodel.add(Flatten())
grandmodel.add(model)

In [102]:
# Plot the model
plot_model(model, to_file='model.png', show_shapes=True)
grandmodel.summary()

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [103]:
opt_adam = optimizers.adam(lr=0.001)
grandmodel.compile(loss='categorical_crossentropy',optimizer=opt_adam,metrics=['accuracy'])
grandmodel.fit(train_data, train_target, batch_size = 32, epochs = 1)

ValueError: Input 0 is incompatible with layer flatten_8: expected min_ndim=3, found ndim=2

Inference Model

In [70]:
# Define encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Define decoder inference model
decoder_state_input_h = Input(shape=embedding_matrix.shape)
decoder_state_input_c = Input(shape=embedding_matrix.shape)
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Summarize model
plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)
plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)

def seq2seq_model_builder():
    
    encoder_inputs = Input(shape=(train_len, ), dtype='int32',)
    encoder_LSTM = LSTM(embedding_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
    decoder_inputs = Input(shape=(train_len, ), dtype='int32',)
    decoder_LSTM = LSTM(embedding_dim, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
    # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)
    
    return model

model = seq2seq_model_builder()
model.summary()

x = Embedding(trainFromSentenceTokens, embedding_dim)
x, state_h = model.layers.GRU(embedding_dim, return_state=True)(x)

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=train_len))
model.add(GRU(32, dropout=0.1, recurrent_dropout=0.5,return_sequences=True))
model.add(GRU(32, dropout=0.1, recurrent_dropout=0.5))

# Load the GloVe embeddings in the model
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,trainFromSentenceTokens))
x = Embedding(trainFromSentenceTokens, embedding_dim)(encoder_inputs)
x, state_h = GRU(embedding_dim, return_state=True)(x)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,trainToSentenceTokens))
x = Embedding(trainToSentenceTokens, embedding_dim)(decoder_inputs)
x = GRU(embedding_dim, return_sequences=True)(x, initial_state=state_h)
decoder_outputs = Dense(trainToSentenceTokens, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([train_data, train_target], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)